In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import numpy as np
import urllib.request

In [208]:
for year in range(1979,1980):
    try:
        filename = f'ftp://ftp2.psl.noaa.gov/Datasets/cpc_global_precip/precip.{year}.nc'
        print(filename)
        dest = f'NOAA/precip.{year}.nc'
        urllib.request.urlretrieve(filename, dest)
    except:
        print("Error:" + filename)


ftp://ftp2.psl.noaa.gov/Datasets/cpc_global_precip/precip.1979.nc


In [225]:
# get data with lat*lon*time rows
ds = xr.open_dataset('NOAA/precip.' + str(year) + '.nc', engine='netcdf4')
df = ds.to_dataframe()
df = df.reset_index()

In [10]:
# read fused loc
fused_loc = pd.read_csv("fused_locations.txt")
fused_loc = fused_loc.rename(columns={'Lat':'lat', 'Lon':'lon'})
fused_loc['fused.loc'] = 1

In [ ]:
# TODO: find interaction: failed -> due to different lon in fused and data
# fused_df = df.merge(fused_loc, on=['lat', 'lon'], how='left')

In [32]:
for year in range(2000,2024):
        # get data with lat*lon*time rows
        ds = xr.open_dataset('NOAA/precip.' + str(year) + '.nc', engine='netcdf4')
        df = ds.to_dataframe()
        df = df.reset_index()

        # convert 0 to 360 latitude to -180 to 180
        df.loc[df['lon'] > 180, 'lon'] -= 360

        # some transformation in Ben's code that I don't understand
        pos_lat = df['lat'] >= 0
        df.loc[pos_lat, 'lat'] = df.loc[pos_lat, 'lat'] - 90
        df.loc[~pos_lat, 'lat'] = df.loc[~pos_lat, 'lat'] + 90
        t1 = df['lat'].copy()
        t2 = df['lon'].copy()
        df['lat'] = -1*t2 + 90
        df['lon'] = -2*t1
        df.loc[df['lat'] > 90, 'lat'] -= 180

        # roughly filter location
        df = df[(df['lat'] >= (min(fused_loc['lat']) - 0.5)) & (df['lat'] <= (max(fused_loc['lat']) - 0.5)) 
                & (df['lon'] >= (min(fused_loc['lon']) - 0.5)) & (df['lon'] <= (max(fused_loc['lon']) - 0.5))]
        
        # Sum to get monthly data
        df['time'] = df['time'].astype(str).str.slice(0, 7)  # drop day value from date
        df = df.groupby(['lat','lon','time']).agg({'precip': lambda x: x.sum(skipna=False)})  # keep NA
        # df = df.groupby(['lat','lon','time']).sum()  # ignore NA
        df = df.reset_index()

        # Merge all into time x location
        lat = min(df['lat'])
        lon = min(df['lon'])
        df_all = df[(df['lat'] == lat) & (df['lon'] == lon)][['time', 'precip']]
        df_all = df_all.rename(columns={'precip': str(lat)+","+str(lon)})

        for lat in np.arange(min(df['lat']),(max(df['lat'])+0.5),0.5):
                for lon in np.arange((min(df['lon'])+1),max(df['lon']+1),1):
                        data_loc = df[(df['lat'] == lat) & (df['lon'] == lon)][['time', 'precip']]
                        data_loc = data_loc.rename(columns={'precip': str(lat)+","+str(lon)})
                        df_all = df_all.merge(data_loc, on='time')

        df_all.to_csv("NOAA/NOAA_" + str(year) + ".csv")